In [3]:
import pandas as pd
import numpy as np
import scipy

import os

In [9]:
df = pd.read_sas('../Data/bid_ask_data.sas7bdat')

In [10]:
df.head()

,PERMNO,DATE,BID,ASK,decile_category
0,10026.0,2000-01-31,18.500,19.1250,8.0
1,10026.0,2000-02-29,17.875,18.5000,8.0
2,10026.0,2000-03-31,19.375,19.8750,8.0
3,10026.0,2000-04-28,15.875,15.9375,8.0
4,10026.0,2000-05-31,14.875,14.9375,8.0
